In [8]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import numpy as np
from sklearn.model_selection import ShuffleSplit, cross_val_score

In [2]:
Jung = pd.read_csv("/home/kumail/Energy_AI/Micro Palstic/Dataset/Jung et Dataset.csv")
Jung = Jung.drop(columns=['Turtle_piece.scan'])
Jung

,identified,4000,3999,3998,3997,3996,3995,3994,3993,3992,...,459,458,457,456,455,454,453,452,451,450
0,HDPE,99.51,99.51,99.51,99.51,99.51,99.50,99.50,99.50,99.50,...,96.83,96.84,96.85,96.83,96.77,96.66,96.54,96.46,96.43,96.44
1,PP,99.03,99.03,99.03,99.03,99.02,99.02,99.02,99.02,99.02,...,93.80,93.77,93.86,94.01,94.09,94.09,94.08,94.12,94.23,94.36
2,PP,99.23,99.22,99.22,99.21,99.21,99.20,99.20,99.21,99.21,...,93.98,94.00,94.05,94.09,94.14,94.23,94.38,94.57,94.75,94.87
3,Mixture,99.50,99.50,99.50,99.50,99.51,99.51,99.51,99.51,99.51,...,96.41,96.45,96.50,96.55,96.62,96.68,96.74,96.77,96.78,96.80
4,HDPE,99.55,99.56,99.56,99.57,99.57,99.57,99.56,99.56,99.56,...,95.28,95.22,95.23,95.30,95.38,95.41,95.35,95.24,95.21,95.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,PP,99.75,99.75,99.75,99.75,99.75,99.75,99.74,99.74,99.73,...,98.58,98.58,98.65,98.76,98.85,98.88,98.87,98.87,98.92,98.98
796,Mixture,99.62,99.62,99.62,99.63,99.63,99.63,99.63,99.63,99.63,...,97.02,97.08,97.16,97.23,97.27,97.30,97.32,97.34,97.39,97.47
797,Mixture,99.92,99.92,99.92,99.92,99.92,99.92,99.92,99.93,99.93,...,96.94,97.01,97.05,97.07,97.08,97.06,97.01,96.97,96.97,97.02
798,HDPE,99.58,99.58,99.58,99.58,99.58,99.58,99.58,99.58,99.57,...,94.32,94.38,94.46,94.51,94.52,94.52,94.53,94.51,94.42,94.31


In [3]:
Jung_counts = Jung['identified'].value_counts()
Jung_counts

identified
HDPE       438
PP         272
Mixture     39
LDPE        37
PS           7
Unknown      4
Nylon        1
PVC          1
PETE         1
Name: count, dtype: int64

In [4]:
values_to_remove = ['Nylon', 'PVC', 'PETE']
Jung = Jung[~Jung['identified'].isin(values_to_remove)]
Jung['identified'] = Jung['identified'].replace('PS', 'Unknown')
Jung_counts = Jung['identified'].value_counts()
Jung_counts


/tmp/ipykernel_783/3446870954.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jung['identified'] = Jung['identified'].replace('PS', 'Unknown')


identified
HDPE       438
PP         272
Mixture     39
LDPE        37
Unknown     11
Name: count, dtype: int64

In [5]:
label_encoder = LabelEncoder()

# Fit and transform the 'Interpretation' column
Jung['identified_encoded'] = label_encoder.fit_transform(Jung['identified'])
Jung = Jung.drop(columns=['identified'])
Jung

/tmp/ipykernel_783/1603336163.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Jung['identified_encoded'] = label_encoder.fit_transform(Jung['identified'])


,4000,3999,3998,3997,3996,3995,3994,3993,3992,3991,...,458,457,456,455,454,453,452,451,450,identified_encoded
0,99.51,99.51,99.51,99.51,99.51,99.50,99.50,99.50,99.50,99.51,...,96.84,96.85,96.83,96.77,96.66,96.54,96.46,96.43,96.44,0
1,99.03,99.03,99.03,99.03,99.02,99.02,99.02,99.02,99.02,99.02,...,93.77,93.86,94.01,94.09,94.09,94.08,94.12,94.23,94.36,3
2,99.23,99.22,99.22,99.21,99.21,99.20,99.20,99.21,99.21,99.22,...,94.00,94.05,94.09,94.14,94.23,94.38,94.57,94.75,94.87,3
3,99.50,99.50,99.50,99.50,99.51,99.51,99.51,99.51,99.51,99.51,...,96.45,96.50,96.55,96.62,96.68,96.74,96.77,96.78,96.80,2
4,99.55,99.56,99.56,99.57,99.57,99.57,99.56,99.56,99.56,99.56,...,95.22,95.23,95.30,95.38,95.41,95.35,95.24,95.21,95.07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,99.75,99.75,99.75,99.75,99.75,99.75,99.74,99.74,99.73,99.73,...,98.58,98.65,98.76,98.85,98.88,98.87,98.87,98.92,98.98,3
796,99.62,99.62,99.62,99.63,99.63,99.63,99.63,99.63,99.63,99.64,...,97.08,97.16,97.23,97.27,97.30,97.32,97.34,97.39,97.47,2
797,99.92,99.92,99.92,99.92,99.92,99.92,99.92,99.93,99.93,99.93,...,97.01,97.05,97.07,97.08,97.06,97.01,96.97,96.97,97.02,2
798,99.58,99.58,99.58,99.58,99.58,99.58,99.58,99.58,99.57,99.57,...,94.38,94.46,94.51,94.52,94.52,94.53,94.51,94.42,94.31,0


In [6]:
X = Jung.drop(columns=['identified_encoded'])
Y = Jung['identified_encoded']

In [9]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(norm='max')
mccv = ShuffleSplit(n_splits=200, test_size=0.2, random_state=42)

clf1 = ExtraTreesClassifier(n_estimators=500, random_state=42)
clf2 = DecisionTreeClassifier(random_state=42)
clf3 = SVC(probability=True, random_state=42)
clf4 = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
clf5 = lgb.LGBMClassifier(random_state=42)
clf6 = RandomForestClassifier(n_estimators=500, random_state=42)
clf9 = GradientBoostingClassifier(random_state=42)
clf10 = AdaBoostClassifier(random_state=42)
clf11 = cb.CatBoostClassifier(silent=True, random_state=42)
clf12 = MLPClassifier(hidden_layer_sizes=(32, 64), activation='relu', solver='adam',
                      max_iter=1000, random_state=42)

# Stacking ensemble
stacking_clf = StackingClassifier(
    estimators=[
         ('et', clf1), ('dt', clf2), ('svc', clf3), ('xgb', clf4), 
        ('lgbm', clf5), ('rf', clf6), ('gb', clf9), 
        ('ab', clf10), ('catboost', clf11), ('mlp', clf12)
    ],
    final_estimator=lgb.LGBMClassifier(random_state=42)
)

In [10]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('normalize', normalizer),
    ('model', stacking_clf)  # Replace 'model' with your actual model, e.g., RandomForestRegressor()
])

In [ ]:
from sklearn.metrics import make_scorer, cohen_kappa_score

# Accuracy
accuracy_scores = cross_val_score(pipeline, X, Y, cv=mccv, scoring='accuracy')
print(f'Average Accuracy: {accuracy_scores.mean():.4f} ± {accuracy_scores.std():.4f}')

# F1 Score (weighted)
f1_scores = cross_val_score(pipeline, X, Y, cv=mccv, scoring='f1_weighted')
print(f'Average F1 Score (Weighted): {f1_scores.mean():.4f} ± {f1_scores.std():.4f}')

# Precision (weighted)
precision_scores = cross_val_score(pipeline, X, Y, cv=mccv, scoring='precision_weighted')
print(f'Average Precision (Weighted): {precision_scores.mean():.4f} ± {precision_scores.std():.4f}')

# Recall (weighted)
recall_scores = cross_val_score(pipeline, X, Y, cv=mccv, scoring='recall_weighted')
print(f'Average Recall (Weighted): {recall_scores.mean():.4f} ± {recall_scores.std():.4f}')

# Cohen's Kappa (requires make_scorer)

kappa_scorer = make_scorer(cohen_kappa_score)
kappa_scores = cross_val_score(pipeline, X, Y, cv=mccv, scoring=kappa_scorer)
print(f'Average Cohen\'s Kappa Score: {kappa_scores.mean():.4f} ± {kappa_scores.std():.4f}')

joblib.dump(pipeline, 'Jung_stacking_model_weights.pkl')

In [26]:
from sklearn.model_selection import cross_val_score, ShuffleSplit, cross_val_predict
from sklearn.metrics import make_scorer, cohen_kappa_score, classification_report
from sklearn.ensemble import ExtraTreesClassifier, StackingClassifier
y_pred = cross_val_predict(pipeline, X, Y, cv=mccv)

# Print classification report
print(classification_report(Y, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.95      0.97      0.96        91
           1       0.57      0.57      0.57         7
           2       1.00      0.60      0.75        10
           3       0.94      0.98      0.96        49
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         3

    accuracy                           0.91       160
   macro avg       0.58      0.52      0.54       160
weighted avg       0.91      0.91      0.91       160



/home/kumail/anaconda3/envs/AliHaq/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kumail/anaconda3/envs/AliHaq/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kumail/anaconda3/envs/AliHaq/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/ho